In [6]:
import torch
import torch.nn as nn

class CountBackbone(nn.Module):
    def __init__(self, input_channels=1, nr_filters=16, kernel_size=3):
        super(CountBackbone, self).__init__()
        # single conv layer for now
        self.conv1 = nn.Conv2d(in_channels=input_channels, 
                        out_channels=nr_filters, 
                        kernel_size=kernel_size, 
                        stride=1, 
                        padding=1)

        # fully connected layer ( for regression ) outputs character count
        self.fc = nn.Linear(nr_filters, 1)

    def forward(self, x):
        # apply convolution
        x = self.conv1(x) 
        # activation function
        x = F.relu(x)
        # global avg pooling
        x = F.adaptive_avg_pool2d(x, (1, 1))
        # flatten for the fuly  connected layer
        x = torch.flatten(x, 1)
        # output 
        x = self.fc(x)





In [7]:
model = CountBackbone()
print(model)

CountBackbone(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=16, out_features=1, bias=True)
)
